In [5]:
import pandas as pd
import numpy as np
import re
from functools import partial
import time

In [6]:
from sklearn.metrics import mean_squared_error

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import  KNeighborsRegressor
from sklearn.svm import LinearSVR, SVR, LinearSVC, SVC, NuSVR, NuSVC
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import clone

In [8]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor, BayesianRidge
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Perceptron, SGDClassifier, PassiveAggressiveClassifier
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, StratifiedKFold, cross_val_predict, RepeatedKFold

from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from tqdm import tqdm
from ax.service.ax_client import AxClient
from xgboost import XGBRegressor, XGBClassifier

In [7]:
class BO_Base:
    def __init__(self, X, y, metric):

        self.X = X
        self.y = y
        self.optimizers = dict()
        self.best_params_ = dict()
        self.best_estimators_ = dict()
        
        for value, keys in self.__getattribute__('metrics').items():
            if metric in keys:
                self.metric = value
        self.metric_alias = self.__getattribute__('metrics')[self.metric][0]

    def _func(self, method, params):
        estimator = clone(
            self.__getattribute__('all_estimators')[method]).set_params(
                **params)

        cv = cross_val_score(estimator,
                             X=self.X,
                             y=self.y,
                             n_jobs=-1,
                             cv=KFold(n_splits=4,
                                      shuffle=True,
                                      random_state=13),
                             scoring=self.metric)
        return {self.metric_alias: (np.mean(cv), np.std(cv))}

    def fit(self, method: str, n_iter: int = 4):
        params = self.__getattribute__('all_params')[method]

        if self.optimizers.get(method, False):
            client = self.optimizers.pop(method)
        else:
            client = AxClient(random_seed=13, verbose_logging=False)
            client.create_experiment(name=f'Optimize {method}',
                                     parameters=params,
                                     objective_name=self.metric_alias,
                                     minimize=False)
        try:
            pb = tqdm(range(n_iter), desc=f'Optimize {method}', leave=True)
            for i in pb:
                parameters, trial_index = client.get_next_trial()
                client.complete_trial(trial_index=trial_index,
                                      raw_data=self._func(method, parameters))
                pb.set_postfix(
                    best=client.get_best_parameters()[1][0][self.metric_alias])
        finally:
            best_parameters, values = client.get_best_parameters()
            self.best_params_[method] = best_parameters
            self.best_estimators_[method] = clone(
                self.__getattribute__('all_estimators')[method]).set_params(
                    **best_parameters)
            self.optimizers[method] = client

In [ ]:
class BO_Clf(BO_Base):
    
    all_params = {
        'log': [{
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'l1_ratio',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }],
        'ridge': [{
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }],
        'lsvm': [{
            'name': 'penalty',
            'type': 'choice',
            'values': ['l1', 'l2'],
            'value_type': 'str',
            }, {
            'name': 'loss',
            'type': 'choice',
            'values': ['hinge', 'squared_hinge'],
            'value_type': 'str',
            }, {
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'intercept_scaling',
            'type': 'range',
            'bounds': [0.01, 10],
            'value_type': 'float',
            }],
        'svm': [{
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'kernel',
            'type': 'choice',
            'values': ['linear', 'poly', 'rbf', 'sigmoid'],
            'value_type': 'str',
            }, {
            'name': 'degree',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'gamma',
            'type': 'choice',
            'values': ['scale', 'auto'],
            'value_type': 'str',
            }, {
            'name': 'coef0',
            'type': 'range',
            'bounds': [-1, 1],
            'value_type': 'float',
            }],
        'nusvm': [{
            'name': 'nu',
            'type': 'range',
            'bounds': [0.01, 1],
            'value_type': 'float',
            }, {
            'name': 'kernel',
            'type': 'choice',
            'values': ['linear', 'poly', 'rbf', 'sigmoid'],
            'value_type': 'str',
            }, {
            'name': 'degree',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'gamma',
            'type': 'choice',
            'values': ['scale', 'auto'],
            'value_type': 'str',
            }, {
            'name': 'coef0',
            'type': 'range',
            'bounds': [-1, 1],
            'value_type': 'float',
            }],
        'passagr': [{
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'loss',
            'type': 'choice',
            'values': ['hinge', 'squared_hinge'],
            'value_type': 'str',
            }],
        'perceptron': [{
            'name': 'penalty',
            'type': 'choice',
            'values': ['l1', 'l2', 'elasticnet'],
            'value_type': 'str',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'eta0',
            'type': 'range',
            'bounds': [0.01, 10],
            'value_type': 'float',
            }],
        'sgd': [{
            'name': 'loss',
            'type': 'choice',
            'values': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
            'value_type': 'str',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'l1_ratio',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'learning_rate',
            'type': 'choice',
            'values': ['constant', 'optimal', 'invscaling', 'adaptive'],
            'value_type': 'str',
            }, {
            'name': 'eta0',
            'type': 'range',
            'bounds': [0.01, 10],
            'value_type': 'float',
            }],
        'rf': [{
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'min_samples_split',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'min_samples_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'max_features',
            'type': 'choice',
            'values': ['auto', 'sqrt', 'log2'],
            'value_type': 'str',
            }],
        'et': [{
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'min_samples_split',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'min_samples_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'max_features',
            'type': 'choice',
            'values': ['auto', 'sqrt', 'log2'],
            'value_type': 'str',
            }],
        'lgbm': [{
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [1e-6, 0.1],
            'value_type': 'float',
            }, {
            'name': 'cross_entropy_lambda',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'rank_xendcg',
            'type': 'range',
            'bounds': [0, 3],
            'value_type': 'int',
            }, {
            'name': 'boosting_type',
            'type': 'choice',
            'values': ['gbdt', 'dart', 'goss'],
            'value_type': 'str',
            }, {
            'name': 'num_leaves',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
            }, {
            'name': 'min_data_in_leaf',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
            }, {
            'name': 'min_sum_hessian_in_leaf',
            'type': 'range',
            'bounds': [1e-6, 0.1],
            'value_type': 'float',
            }, {
            'name': 'bagging_fraction',
            'type': 'range',
            'bounds': [0.5, 1],
            'value_type': 'float',
            }, {
            'name': 'bagging_frec',
            'type': 'range',
            'bounds': [1, 10],
            'value_type': 'int',
            }, {
            'name': 'feature_fraction',
            'type': 'range',
            'bounds': [0.5, 1],
            'value_type': 'float',
            }],
        'cat': [{
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [1e-6, 0.1],
            'value_type': 'float',
            }, {
            'name': 'num_leaves',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
            }, {
            'name': 'l2_leaf_reg',
            'type': 'range',
            'bounds': [1e-2, 10],
            'value_type': 'float',
#             }, {
#             'name': 'grow_policy',
#             'type': 'fixed',
#             'values': ['Depthwise', 'SymmetricTree', 'Lossguide'],
#             'value_type': 'str',
            }, {
            'name': 'min_data_in_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'colsample_bylevel',
            'type': 'range',
            'bounds': [0.1, 1],
            'value_type': 'float',
            }, {
            'name': 'od_pval',
            'type': 'range',
            'bounds': [10e-10, 10e-2],
            'value_type': 'float',
            }, {
            'name': 'langevin',
            'type': 'choice',
            'values': [True, False],
            'value_type': 'bool',
            }],
        'xgb': [{
#             'name': 'booster',
#             'type': 'choice',
#             'values': ['gbtree', 'gblinear', 'dart'],
#             'value_type': 'str',
#             }, {
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [1e-06, 0.1],
            'value_type': 'float',
            }, {
            'name': 'gamma',
            'type': 'range',
            'bounds': [0.01, 1],
            'value_type': 'float',
            }, {
            'name': 'min_child_weight',
            'type': 'range',
            'bounds': [0.01, 2],
            'value_type': 'float',
            }, {
            'name': 'subsample',
            'type': 'range',
            'bounds': [0.5, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bytree',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bylevel',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bynode',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'reg_alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'reg_lambda',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }],
    }

    all_estimators = {
        'log': LogisticRegression(penalty='elasticnet',
                                  solver='saga',
                                  class_weight='balanced',
                                  random_state=13, 
                                  n_jobs=-1),
        'ridge': RidgeClassifier(solver='saga',
                                 class_weight='balanced',
                                 max_iter=100,
                                 random_state=13),
        'lsvm': LinearSVC(dual=False,
                          class_weight='balanced',
                          max_iter=100, 
                          random_state=13),
        'svm': SVC(max_iter=1000,
                   class_weight='balanced',
                   random_state=13),
        'nusvm': NuSVC(max_iter=100,
                       class_weight='balanced',
                       random_state=13),
        'passagr': PassiveAggressiveClassifier(max_iter=100,
                                               early_stopping=True,
                                               n_iter_no_change=10,
                                               n_jobs=-1,
                                               random_state=13),
        'perceptron': Perceptron(max_iter=100,
                                 n_jobs=-1,
                                 random_state=13,
                                 n_iter_no_change=10,
                                early_stopping=True),
        'sgd': SGDClassifier(penalty='elasticnet',
                             n_jobs=-1,
                             random_state=13,
                             n_iter_no_change=10,
                             early_stopping=True),
        'rf': RandomForestClassifier(class_weight='balanced_subsample',
                                     bootstrap=True,
                                     n_jobs=-1,
                                     random_state=13),
        'et': ExtraTreesClassifier(class_weight='balanced_subsample',
                                   bootstrap=True,
                                   n_estimators=100,
                                   n_jobs=-1,
                                   random_state=13),
        'lgbm':
        LGBMClassifier(tree_learner='voting',
                       n_estimators=1000,
                       random_state=13,
                       n_jobs=0),
        'cat':
        CatBoostClassifier(grow_policy='Lossguide',
                           loss_function='CrossEntropy',
                           n_estimators=100,
                          random_state=13,
                          early_stopping_rounds=10),
        'xgb': XGBClassifier(booster='dart',
                             tree_method='hist',
                             n_estimators=100,
                             use_label_encoder=False,
                             n_jobs=-1,
                            verbosity=1,
                            random_state=13),
    }
    
    metrics = {'roc_auc': ['ROC_AUC', 'roc_auc', 'rocauc'],
               'accuracy': ['Accuracy', 'Acc', 'acc', 'accuracy']}
        
    def __init__(self, X, y, metric='ROC_AUC'):
        BO_Base.__init__(self, X, y, metric)
        
        for m in ['xgb']:
            BO_Clf.all_estimators[m].set_params(eval_metric='auc' if self.metric == 'roc_auc' else 'logloss',
                                               scale_pos_weight=(len(y)-sum(y))/sum(y) if self.metric == 'roc_auc' else 1)

In [156]:
class BO_Reg(BO_Base):
    
    all_params = {
        'ridge': [{
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 50],
            'value_type': 'float',
            }],
        'kridge': [{
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'kernel',
            'type': 'choice',
            'values': ['polynomial', 'rbf', 'laplacian',
                       'sigmoid', 'cosine', 'linear',],
            'value_type': 'str',
            }, {
            'name': 'gamma',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'degree',
            'type': 'range',
            'bounds': [1, 10],
            'value_type': 'int',
            }, {
            'name': 'coef0',
            'type': 'range',
            'bounds': [0.01, 1],
            'value_type': 'float',
            }],
        'lasso': [{
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'selection',
            'type': 'choice',
            'values': ['cyclic', 'random'],
            'value_type': 'str',
            }],
        'elastic': [{
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 50],
            'value_type': 'float',
            }, {
            'name': 'l1_ratio',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'selection',
            'type': 'choice',
            'values': ['cyclic', 'random'],
            'value_type': 'str',
            }],
        'huber': [{
            'name': 'epsilon',
            'type': 'range',
            'bounds': [1.01, 2],
            'value_type': 'float',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }],
        'bayes': [{
            'name': 'alpha_1',
            'type': 'range',
            'bounds': [10e-10, 0.1],
            'value_type': 'float',
            }, {
            'name': 'alpha_2',
            'type': 'range',
            'bounds': [10e-10, 0.1],
            'value_type': 'float',
            }, {
            'name': 'lambda_1',
            'type': 'range',
            'bounds': [10e-10, 0.1],
            'value_type': 'float',
            }, {
            'name': 'lambda_2',
            'type': 'range',
            'bounds': [10e-10, 0.1],
            'value_type': 'float',
            }],
        'lsvm': [{
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'loss',
            'type': 'choice',
            'values': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
            'value_type': 'str',
            }, {
            'name': 'intercept_scaling',
            'type': 'range',
            'bounds': [0.01, 10],
            'value_type': 'float',
            }],
        'svm': [{
            'name': 'C',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'kernel',
            'type': 'choice',
            'values': ['linear', 'poly', 'rbf', 'sigmoid'],
            'value_type': 'str',
            }, {
            'name': 'degree',
            'type': 'range',
            'bounds': [2, 10],
            'value_type': 'int',
            }, {
            'name': 'gamma',
            'type': 'choice',
            'values': ['scale', 'auto'],
            'value_type': 'str',
            }, {
            'name': 'coef0',
            'type': 'range',
            'bounds': [-1, 1],
            'value_type': 'float',
            }, {
            'name': 'epsilon',
            'type': 'range',
            'bounds': [0.01, 1],
            'value_type': 'float',
            }],
        'sgd': [{
            'name': 'loss',
            'type': 'choice',
            'values': ['epsilon_insensitive', 'huber', 'squared_epsilon_insensitive', 'squared_loss'],
            'value_type': 'str',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'l1_ratio',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'learning_rate',
            'type': 'choice',
            'values': ['constant', 'optimal', 'invscaling', 'adaptive'],
            'value_type': 'str',
            }, {
            'name': 'eta0',
            'type': 'range',
            'bounds': [0.01, 10],
            'value_type': 'float',            
            }],
        'gbm': [{
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'loss',
            'type': 'choice',
            'values': ['ls', 'lad', 'huber', 'quantile'],
            'value_type': 'str',
            }, {
            'name': 'subsample',
            'type': 'range',
            'bounds': [0.001, 1],
            'value_type': 'float',
            }, {
            'name': 'min_samples_split',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'min_samples_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
            }, {
            'name': 'max_features',
            'type': 'choice',
            'values': ['auto', 'sqrt', 'log2'],
            'value_type': 'str',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [0.001, 0.999],
            'value_type': 'float',
            }, {
            'name': 'ccp_alpha',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }],
        'lgbm': [{
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
#             }, {
#             'name': 'bagging_fraction',
#             'type': 'range',
#             'bounds': [0.5, 1],
#             'value_type': 'float',
#             }, {
#             'name': 'bagging_frec',
#             'type': 'range',
#             'bounds': [1, 10],
#             'value_type': 'int',
#             }, {
#             'name': 'feature_fraction',
#             'type': 'range',
#             'bounds': [0.5, 1],
#             'value_type': 'float',
#             }, {
            'name': 'boosting_type',
            'type': 'choice',
            'values': ['gbdt', 'dart', 'goss'],
            'value_type': 'str',
            }, {
            'name': 'num_leaves',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
            }, {
            'name': 'min_data_in_leaf',
            'type': 'range',
            'bounds': [2, 100],
            'value_type': 'int',
#             }, {
#             'name': 'reg_alpha',
#             'type': 'range',
#             'bounds': [0, 1],
#             'value_type': 'float',
#             }, {
#             'name': 'reg_lambda',
#             'type': 'range',
#             'bounds': [0, 1],
#             'value_type': 'float',
#             }, {
#             'name': 'extra_trees',
#             'type': 'choice',
#             'values': [True, False],
#             'value_type': 'bool',
            }],
        'cat': [{
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'l2_leaf_reg',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'bootstrap_type',
            'type': 'choice',
            'values': ['Bayesian', 'Bernoulli', 'MVS'],
            'value_type': 'str',
            }, {
            'name': 'grow_policy',
            'type': 'choice',
            'values': ['SymmetricTree', 'Depthwise', 'Lossguide'],
            'value_type': 'str',
            }, {
            'name': 'min_data_in_leaf',
            'type': 'range',
            'bounds': [1, 100],
            'value_type': 'int',
            }, {
            'name': 'colsample_bylevel',
            'type': 'range',
            'bounds': [0.01, 1],
            'value_type': 'float',
            }, {
            'name': 'langevin',
            'type': 'choice',
            'values': [True, False],
            'value_type': 'bool',
            }, {
            'name': 'penalties_coefficient',
            'type': 'range',
            'bounds': [10e-5, 10],
            'value_type': 'float',
            }, {
            'name': 'od_pval',
            'type': 'range',
            'bounds': [10e-10, 10e-2],
            'value_type': 'float',
            }],
        'xgb': [{
            'name': 'booster',
            'type': 'choice',
            'values': ['gbtree', 'gblinear', 'dart'],
            'value_type': 'str',
            }, {
            'name': 'max_depth',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'learning_rate',
            'type': 'range',
            'bounds': [0.00001, 0.1],
            'value_type': 'float',
            }, {
            'name': 'gamma',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'min_child_weight',
            'type': 'range',
            'bounds': [0, 2],
            'value_type': 'float',
            }, {
            'name': 'subsample',
            'type': 'range',
            'bounds': [0.5, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bytree',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bylevel',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'colsample_bynode',
            'type': 'range',
            'bounds': [0.8, 1],
            'value_type': 'float',
            }, {
            'name': 'reg_alpha',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'reg_lambda',
            'type': 'range',
            'bounds': [0, 10],
            'value_type': 'float',
            }],
        'rf': [{
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'min_samples_split',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'min_samples_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'max_features',
            'type': 'choice',
            'values': ['auto', 'sqrt', 'log2'],
            'value_type': 'str',
            }],
        'et': [{
            'name': 'max_depth',
            'type': 'range',
            'bounds': [2, 50],
            'value_type': 'int',
            }, {
            'name': 'min_samples_split',
            'type': 'range',
            'bounds': [2, 20],
            'value_type': 'int',
            }, {
            'name': 'min_samples_leaf',
            'type': 'range',
            'bounds': [1, 20],
            'value_type': 'int',
            }, {
            'name': 'max_features',
            'type': 'choice',
            'values': ['auto', 'sqrt', 'log2'],
            'value_type': 'str',
            }],
        'mlp': [{
            'name': 'activation',
            'type': 'choice',
            'values': ['identity', 'logistic', 'tanh', 'relu'],
            'value_type': 'str',
            }, {
            'name': 'solver',
            'type': 'choice',
            'values': ['lbfgs', 'adam'],
            'value_type': 'str',
            }, {
            'name': 'alpha',
            'type': 'range',
            'bounds': [1e-6, 10],
            'value_type': 'float',
            }, {
            'name': 'learning_rate',
            'type': 'choice',
            'values': ['constant', 'invscaling', 'adaptive'],
            'value_type': 'str',
            }, {
            'name': 'momentum',
            'type': 'range',
            'bounds': [0, 1],
            'value_type': 'float',
            }, {
            'name': 'beta_1',
            'type': 'range',
            'bounds': [0, 0.999],
            'value_type': 'float',
            }, {
            'name': 'beta_2',
            'type': 'range',
            'bounds': [0, 0.999],
            'value_type': 'float',
        }]
    }

    all_estimators = {
        'ridge':
        Ridge(solver='saga',
              max_iter=100000,
              random_state=13),
        'kridge':
        KernelRidge(),
        'lasso':
        Lasso(max_iter=100000,
              random_state=13),
        'huber':
        HuberRegressor(max_iter=1000),
        'elastic':
        ElasticNet(max_iter=1000,
                   random_state=13),
        'bayes':
        BayesianRidge(n_iter=100000),
        'lsvm': LinearSVR(dual=False,
                          max_iter=100,
                          random_state=13),
        'svm': SVR(max_iter=10000),
        'sgd': SGDClassifier(penalty='elasticnet',
                             n_jobs=-1,
                             random_state=13,
                             n_iter_no_change=10,
                             early_stopping=True),
        'gbm':
        GradientBoostingRegressor(n_estimators=10000,
                                  random_state=13,
                                  n_iter_no_change=10),
        'lgbm':
        LGBMRegressor(n_estimators=100,
                      random_state=13,
                      n_jobs=-1),
        'cat':
        CatBoostRegressor(n_estimators=100,
                          random_state=13,
                          early_stopping_rounds=10),
        'xgb': XGBRegressor(n_estimators=100,
                            verbosity=0,
                            n_jobs=-1,
                            random_state=13),
        'rf':
        RandomForestRegressor(bootstrap=True,
                              n_estimators=100,
                              n_jobs=-1,
                              random_state=13),
        'et': ExtraTreesRegressor(bootstrap=True,
                                  n_estimators=100,
                                  n_jobs=-1,
                                  random_state=13),
        'mlp':
        MLPRegressor(max_iter=100,
                     early_stopping=True,
                     n_iter_no_change=10,
                     random_state=13)
    }
    
    metrics = {'neg_root_mean_squared_error': ['RMSE', 'rmse'],
               'neg_mean_absolute_error': ['MAE', 'mae']}
    
    def __init__(self, X, y, metric='RMSE'):
        BO_Base.__init__(self, X, y, metric)
        
        for m in ['rf', 'et']:
            BO_Reg.all_estimators[m].set_params(criterion='mse' if self.metric == 'neg_root_mean_squared_error' else 'mae')        

In [ ]:
class BO_blend:
    def __init__(self, X, y):

        self.X = X
        self.y = y

        self.parameters = []
        self.constrains = []
        constrain = X.columns[0]
        for col in X.columns:
            p = dict()
            p['name'] = col
            p['type'] = 'range'
            p['bounds'] = [0, 1]
            p['value_type'] = 'float'
            self.parameters.append(p)
            if col != X.columns[0]:
                constrain += f' + {col}'
        self.constrains.append(constrain + ' <= 1.01')
        self.constrains.append(constrain + ' >= 0.99')

        self.client = AxClient(random_seed=13, verbose_logging=True)
        self.client.create_experiment(name='Optimizing coefs for voting',
                                      parameters=self.parameters,
                                      objective_name='RMSE',
                                      minimize=True,
                                      parameter_constraints=self.constrains)

    def __func(X, y, prms):

        rmse = []
        for i, _ in RepeatedKFold(n_splits=4, n_repeats=2,
                                  random_state=13).split(y):
            rez = pd.Series(data=0, index=i)
            for col, coef in prms.items():
                rez += X[col].iloc[i].values * coef

            rmse.append(mean_squared_error(y.iloc[i], rez, squared=False))

        return ({'RMSE': (np.mean(rmse), np.std(rmse))})

    def fit(self, n_iter=50):
        try:
            for i in tqdm(range(n_iter)):
                parameters, trial_index = self.client.get_next_trial()
                self.client.complete_trial(trial_index=trial_index,
                                           raw_data=BO_blend.__func(
                                               X=self.X,
                                               y=self.y,
                                               prms=parameters))

        finally:
            self.best_parameters, values = self.client.get_best_parameters()
            print(values[0])
            return (self.best_parameters)